<a href="https://colab.research.google.com/github/beekiran00/DeepMirror-Task-ML-Engineer---Bhanu-Velpula/blob/main/Graph_Neural_Network_DM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading and Importing Libraries

In [4]:
!pip install rdkit
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html
!pip install PyTDC
!pip install deepchem
!pip install dgl
!pip install dgllife
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cpu.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 512 kB 4.4 MB/s 


In [5]:
# IMPORT THE NECESSARY LIBRARIES

# BASIC PANDAS AND NUMPY, matplotlib libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch_geometric.nn import global_mean_pool
from IPython.display import Javascript
import time

# RDkit
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix

# DEEPCHEM
import deepchem as dc
from deepchem.models import GraphConvModel

# Tensorflow
from tensorflow.python import train

# SKLEARN and sklearn metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score

from sklearn.metrics import confusion_matrix



# Pytorch and Pytorch Geometric
import torch
from torch_geometric.data import Data
from torch.utils.data import DataLoader
from torch_geometric.loader import DataLoader
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

from sklearn.metrics import confusion_matrix
from torchmetrics.classification import AUROC
from torchmetrics.classification import ROC
from torchmetrics.classification import Precision
from torchmetrics.classification import Specificity



# Datasets

In [6]:
from tdc.single_pred import ADME
data = ADME(name = 'HIA_Hou')
split = data.get_split()

Downloading...
100%|██████████| 40.1k/40.1k [00:00<00:00, 10.8MiB/s]
Loading...
Done!


Viewing the dataset, the dataset is in the form of a dictionary, train: values, valid: values and test: values

In [7]:
#split

In [8]:
print("Dataset type: ", type(data))
print(split.keys())
#print(split.values())


Dataset type:  <class 'tdc.single_pred.adme.ADME'>
dict_keys(['train', 'valid', 'test'])


In [9]:
train_df = pd.DataFrame.from_dict(split['train'])
train_df
valid_df = pd.DataFrame.from_dict(split['valid'])
#valid_df
test_df = pd.DataFrame.from_dict(split['test'])
#test_df

# ^ converting the dict to a dataframe of train valid and test

# Graph Neural Network Class

## SMILES to GRAPH


Since the dataset only has the SMILES String under the column 'Drug', we have to extract features fromthe SMILES String, only problem is, unlike normal mehtods for feature extraction from SMILES, graph neural networks takes a graph structure, and this is a graph level task. Therefore, the firest step would be to take the dataset as input, and convert them to graph data

In summary, the functions below converts a given smiles string, into a graph data format, which is used in Pytorch GNN model.

In [10]:
def one_hot_encoding(x, permitted_list):
    """
    Maps input elements x which are not in the permitted list to the last element
    of the permitted list.
    """

    if x not in permitted_list:
        x = permitted_list[-1]

    binary_encoding = [int(boolean_value) for boolean_value in list(map(lambda s: x == s, permitted_list))]

    return binary_encoding

In [11]:
def get_atom_features(atom, 
                      use_chirality = True, 
                      hydrogens_implicit = True):
    """
    Takes an RDKit atom object as input and gives a 1d-numpy array of atom features as output.
    """

    # define list of permitted atoms
    
    permitted_list_of_atoms =  ['C','N','O','S','F','Si','P','Cl','Br','Mg','Na','Ca','Fe','As','Al','I', 'B','V','K','Tl','Yb','Sb','Sn','Ag','Pd','Co','Se','Ti','Zn', 'Li','Ge','Cu','Au','Ni','Cd','In','Mn','Zr','Cr','Pt','Hg','Pb','Unknown']
    
    if hydrogens_implicit == False:
        permitted_list_of_atoms = ['H'] + permitted_list_of_atoms
    
    # compute atom features
    
    atom_type_enc = one_hot_encoding(str(atom.GetSymbol()), permitted_list_of_atoms)
    
    n_heavy_neighbors_enc = one_hot_encoding(int(atom.GetDegree()), [0, 1, 2, 3, 4, "MoreThanFour"])
    
    formal_charge_enc = one_hot_encoding(int(atom.GetFormalCharge()), [-3, -2, -1, 0, 1, 2, 3, "Extreme"])
    
    hybridisation_type_enc = one_hot_encoding(str(atom.GetHybridization()), ["S", "SP", "SP2", "SP3", "SP3D", "SP3D2", "OTHER"])
    
    is_in_a_ring_enc = [int(atom.IsInRing())]
    
    is_aromatic_enc = [int(atom.GetIsAromatic())]
    
    atomic_mass_scaled = [float((atom.GetMass() - 10.812)/116.092)]
    
    vdw_radius_scaled = [float((Chem.GetPeriodicTable().GetRvdw(atom.GetAtomicNum()) - 1.5)/0.6)]
    
    covalent_radius_scaled = [float((Chem.GetPeriodicTable().GetRcovalent(atom.GetAtomicNum()) - 0.64)/0.76)]

    atom_feature_vector = atom_type_enc + n_heavy_neighbors_enc + formal_charge_enc + hybridisation_type_enc + is_in_a_ring_enc + is_aromatic_enc + atomic_mass_scaled + vdw_radius_scaled + covalent_radius_scaled
                                    
    if use_chirality == True:
        chirality_type_enc = one_hot_encoding(str(atom.GetChiralTag()), ["CHI_UNSPECIFIED", "CHI_TETRAHEDRAL_CW", "CHI_TETRAHEDRAL_CCW", "CHI_OTHER"])
        atom_feature_vector += chirality_type_enc
    
    if hydrogens_implicit == True:
        n_hydrogens_enc = one_hot_encoding(int(atom.GetTotalNumHs()), [0, 1, 2, 3, 4, "MoreThanFour"])
        atom_feature_vector += n_hydrogens_enc

    return np.array(atom_feature_vector)

In [12]:
def get_bond_features(bond, 
                      use_stereochemistry = True):
    """
    Takes an RDKit bond object as input and gives a 1d-numpy array of bond features as output.
    """

    permitted_list_of_bond_types = [Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE, Chem.rdchem.BondType.TRIPLE, Chem.rdchem.BondType.AROMATIC]

    bond_type_enc = one_hot_encoding(bond.GetBondType(), permitted_list_of_bond_types)
    
    bond_is_conj_enc = [int(bond.GetIsConjugated())]
    
    bond_is_in_ring_enc = [int(bond.IsInRing())]
    
    bond_feature_vector = bond_type_enc + bond_is_conj_enc + bond_is_in_ring_enc
    
    if use_stereochemistry == True:
        stereo_type_enc = one_hot_encoding(str(bond.GetStereo()), ["STEREOZ", "STEREOE", "STEREOANY", "STEREONONE"])
        bond_feature_vector += stereo_type_enc

    return np.array(bond_feature_vector)

In [13]:
def create_pytorch_geometric_graph_data_list_from_smiles_and_labels(x_smiles, y):
    """
    Inputs:
    
    x_smiles = [smiles_1, smiles_2, ....] ... a list of SMILES strings
    y = [y_1, y_2, ...] ... a list of numerial labels for the SMILES strings (such as associated pKi values)
    
    Outputs:
    
    data_list = [G_1, G_2, ...] ... a list of torch_geometric.data.Data objects which represent labeled molecular graphs that can readily be used for machine learning
    
    """
    
    data_list = []
    
    for (smiles, y_val) in zip(x_smiles, y):
        
        # convert SMILES to RDKit mol object
        mol = Chem.MolFromSmiles(smiles)

        # get feature dimensions
        n_nodes = mol.GetNumAtoms()
        n_edges = 2*mol.GetNumBonds()
        unrelated_smiles = "O=O"
        unrelated_mol = Chem.MolFromSmiles(unrelated_smiles)
        n_node_features = len(get_atom_features(unrelated_mol.GetAtomWithIdx(0)))
        n_edge_features = len(get_bond_features(unrelated_mol.GetBondBetweenAtoms(0,1)))

        # construct node feature matrix X of shape (n_nodes, n_node_features)
        X = np.zeros((n_nodes, n_node_features))

        for atom in mol.GetAtoms():
            X[atom.GetIdx(), :] = get_atom_features(atom)
            
        X = torch.tensor(X, dtype = torch.float)
        
        # construct edge index array E of shape (2, n_edges)
        (rows, cols) = np.nonzero(GetAdjacencyMatrix(mol))
        torch_rows = torch.from_numpy(rows.astype(np.int64)).to(torch.long)
        torch_cols = torch.from_numpy(cols.astype(np.int64)).to(torch.long)
        E = torch.stack([torch_rows, torch_cols], dim = 0)
        
        # construct edge feature array EF of shape (n_edges, n_edge_features)
        EF = np.zeros((n_edges, n_edge_features))
        
        for (k, (i,j)) in enumerate(zip(rows, cols)):
            
            EF[k] = get_bond_features(mol.GetBondBetweenAtoms(int(i),int(j)))
        
        EF = torch.tensor(EF, dtype = torch.float)
        
        # construct label tensor
        y_tensor = torch.tensor(np.array([y_val]), dtype = torch.float)
        
        # construct Pytorch Geometric data object and append to data list
        data_list.append(Data(x = X, edge_index = E, edge_attr = EF, y = y_tensor))

    return data_list

Now that we've written our functions, the next step is to pass the datasets to turn them into graphs

In [14]:
# turn train dataset into a graph
train_dataset = create_pytorch_geometric_graph_data_list_from_smiles_and_labels(train_df["Drug"], train_df["Y"])
#turn valid dataset into a graph
valid_dataset = create_pytorch_geometric_graph_data_list_from_smiles_and_labels(valid_df["Drug"], valid_df["Y"])
#turn test dataset into a graph
test_dataset = create_pytorch_geometric_graph_data_list_from_smiles_and_labels(test_df["Drug"], test_df["Y"])

In [15]:
# The output of the dataset is in terms of a list, but the values inside the list are graph objects.
type(train_dataset)

list

In [16]:
# load the data using pytorch dataloader

# Data loader. Combines a dataset and a sampler, and provides an iterable over the given dataset
graph_train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True) # train dataset into dataloader
graph_val_loader = DataLoader(valid_dataset, batch_size=64) # Additional loader if you want to change to a larger dataset
graph_test_loader = DataLoader(test_dataset, batch_size=64) # test dataset into dataloader

In [17]:
print(graph_test_loader) # we have a torch object

In [18]:
batch = next(iter(graph_test_loader))
print("Batch:", batch)
print("Labels:", batch.y[:10])
print("Batch indices:", batch.batch[:40])

Batch: DataBatch(x=[1466, 79], edge_index=[2, 3124], edge_attr=[3124, 10], y=[64], batch=[1466], ptr=[65])
Labels: tensor([1., 0., 1., 1., 1., 1., 1., 1., 0., 1.])
Batch indices: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [19]:
# view the number of graphs and the batch size
for step, data in enumerate(graph_train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(x=[1384, 79], edge_index=[2, 2980], edge_attr=[2980, 10], y=[64], batch=[1384], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(x=[1498, 79], edge_index=[2, 3192], edge_attr=[3192, 10], y=[64], batch=[1498], ptr=[65])

Step 3:
Number of graphs in the current batch: 64
DataBatch(x=[1567, 79], edge_index=[2, 3368], edge_attr=[3368, 10], y=[64], batch=[1567], ptr=[65])

Step 4:
Number of graphs in the current batch: 64
DataBatch(x=[1549, 79], edge_index=[2, 3332], edge_attr=[3332, 10], y=[64], batch=[1549], ptr=[65])

Step 5:
Number of graphs in the current batch: 64
DataBatch(x=[1337, 79], edge_index=[2, 2864], edge_attr=[2864, 10], y=[64], batch=[1337], ptr=[65])

Step 6:
Number of graphs in the current batch: 64
DataBatch(x=[1476, 79], edge_index=[2, 3186], edge_attr=[3186, 10], y=[64], batch=[1476], ptr=[65])

Step 7:
Number of graphs in the current batch: 20
DataBatch(x=[488, 79], edge_index=[2, 1038],

In [20]:
# combine the dataset to get the number of features, this input goes into the pytorch GNN model later
dataset = []
dataset.append(train_dataset)
dataset.append(valid_dataset)
dataset.append(test_dataset)


In [21]:
# Compute the num_node_features, since the graph data is inside a list of list
sum=0
for i in dataset:
  for j in i:
    num = j.num_node_features
    print(num)
    sum +=num
print(sum)

# the num_node features is 79
    

79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
79
7

In [22]:

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(79, hidden_channels) # define first convolution
        self.conv2 = GCNConv(hidden_channels, hidden_channels) # define the second convolution
        self.conv3 = GCNConv(hidden_channels, hidden_channels) # define the third convolution
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x
# call the model and print the model, its convolutions
model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(79, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [23]:
# misc to display output
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

# call the model class we created for the Graph Neural Networks
model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) # initalise the optimiser, we used ADAM optimiser with learing rate lr = 0.01
criterion = torch.nn.CrossEntropyLoss() #initialise the criterion, CrossEntropyLoss used here

# define the train and test models

def train():
    model.train()

    for data in graph_train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y.type(torch.LongTensor))  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()
     CM=0

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum()) 
         auroc = AUROC(task="binary")
         au_roc_score = auroc(pred, data.y) # gets the roc_auc score of the model
         precision = Precision(task="binary", average='macro', num_classes=3)
         precision_score = precision(pred, data.y) # gets the precision of the model
         specificity = Specificity(task="binary", average='macro', num_classes=3)
         spec_score = specificity(pred, data.y) # gets hte specificity of the model
         
         
            
     return correct / len(loader.dataset), au_roc_score, precision_score, spec_score # Derive ratio of correct predictions.

start_time = time.time()
# For loop to train and test the model for 170 epochs
for epoch in range(1, 171):
   
    train()
    train_acc = test(graph_train_loader)
    test_acc = test(graph_test_loader)

    # print the epoch number and metrics
    print(f'Epoch: {epoch:03d}',train_acc, test_acc)
print("--- %s seconds ---" % (time.time() - start_time))

<IPython.core.display.Javascript object>

Epoch: 001 (0.8688118811881188, tensor(0.5000), tensor(0.9500), tensor(0.)) (0.8362068965517241, tensor(0.5000), tensor(0.8077), tensor(0.))
Epoch: 002 (0.8688118811881188, tensor(0.5000), tensor(0.7500), tensor(0.)) (0.8362068965517241, tensor(0.5000), tensor(0.8077), tensor(0.))
Epoch: 003 (0.8688118811881188, tensor(0.5000), tensor(0.9500), tensor(0.)) (0.8362068965517241, tensor(0.5000), tensor(0.8077), tensor(0.))
Epoch: 004 (0.8688118811881188, tensor(0.5000), tensor(0.9500), tensor(0.)) (0.8362068965517241, tensor(0.5000), tensor(0.8077), tensor(0.))
Epoch: 005 (0.8910891089108911, tensor(0.5000), tensor(0.9500), tensor(0.)) (0.8706896551724138, tensor(0.6500), tensor(0.8571), tensor(0.3000))
Epoch: 006 (0.8935643564356436, tensor(0.6250), tensor(0.8421), tensor(0.2500)) (0.8706896551724138, tensor(0.6500), tensor(0.8571), tensor(0.3000))
Epoch: 007 (0.8910891089108911, tensor(0.7222), tensor(0.9444), tensor(0.5000)) (0.9051724137931034, tensor(0.7000), tensor(0.8750), tensor(0.